# Regular Languages and Automata


**Objectives:**
1. Familiarize with graph representation of finite automata. 

2. Implement basic operations on finite automaton. 

3. Practice elementary proof techniques.


**Reading:** Chap. 1. 


**Questions:**
1. What is a state in finite automaton? What is a `dead` state? In our work, we generally refer to a dead state as a `sink` state.

2. What is an alphabet? What is a string? What is a language?

3. What is an incomplete automaton? How to complete an incomplete automaton?

4. What is a regular language? What other types of languages exist? 